# Opposite side tagging (OS) tagging

## Data: 

* real data $B^+ \to J\psi K^+$, $B^- \to J\psi K^-$ 
* apply sPlot to obtain sWeight ~ P(B)
* set `event_id` - EventNumber & RunNumber

## Tracking inclusive OS tagging



### Selections

* Input: all possible tracks for all B-events.
* (PIDNNp < tr) & (PIDNNpi < tr) & (ghostProb < 0.4), tr=0.6
* ((PIDNNk > trk) | (PIDNNm > trm) | (PIDNNe > tre)), trk=0.7, trm=0.4, tre=0.6



### $\epsilon_{tag}$ calculation

$$N (\text{passed selection}) = \sum_{\text{passed selection}} sw_i$$

$$N (\text{all events}) = \sum_{\text{all events}} sw_i,$$

where $sw_i$ - sPLot weight

$$\epsilon_{tag} = \frac{N (\text{passed selection})} {N (\text{all events})}$$

$$\Delta\epsilon_{tag} = \frac{\sqrt{N (\text{passed selection})}} {N (\text{all events})}$$


### Data for training

* `data_sw_passed` - tracks with **B-sWeight > 1**, are used for training
* `data_sw_not_passed` - tracks with **B-sWeight <= 1**, are tagged after training

TODO: add plots

### Training 

#### Features:

* `cos_diff_phi` 
* `diff_pt`
* `partPt`
* `max_PID_e_mu`
* `partP`
* `nnkrec`
* `diff_eta`
* `EOverP`
* `sum_PID_k_mu`
* `ptB`
* `sum_PID_e_mu`
* `sum_PID_k_e`
* `proj`
* `PIDNNe`
* `PIDNNk`
* `PIDNNm`
* `phi`
* `IP`
* `max_PID_k_mu`
* `IPerr`
* `IPs`
* `ID`
* `veloch`
* `max_PID_k_e`
* `diff_phi`
* `ghostProb`
* `IPPU`
* `eta`
* `partlcs`

#### Classifier

Try to define B sign using track sign (to define they have the same signs or opposite).

`target` = `signB` * `signTrack` > 0

* classifier returns $P(\text{track same sign as B| B sign}) = P(\text{B same sign as track| track sign})$
* 2-folding training on the full training sample to use full sample for futher analysis (folding scheme provides not overfitted model, details: http://yandex.github.io/rep/metaml.html#module-rep.metaml.folding)

#### Calibration of $P(\text{track same sign as B| B sign})$

* use 2-folding logistic calibration for track classifier's prediction
* compare with isotonic calibration (bad)
* compare with absent calibration (bad, have shift predictions)


#### Computation of $p(B^+)$ using $P(\text{track same sign as B| B sign})$

Compute $p(B^+)$ using this probabilistic model representation (similar to previous tagging combination):

$$ \frac{P(B^+)}{P(B^-)} = \prod_{track} \frac{P(\text{track/vertex}|B^+)} {P(\text{track/vertex} |B^-)} = P
\qquad \Rightarrow \qquad P(B^+) = \frac {P}{1+P}, $$
where
$$ P(\text{track/vertex}^+|B^+) = P(\text{track/vertex same sign as } B| B) $$
$$ P(\text{track/vertex}^+|B^-) = 1 - P(\text{track/vertex same sign as } B| B)$$
$$ P(\text{track/vertex}^-|B^+) = 1 - P(\text{track/vertex same sign as }B| B) $$
$$ P(\text{track/vertex}^-|B^-) = P(\text{track/vertex same sign as } B| B)$$


#### Intermediate estimation $<D^2>$ for tracking

Do calibration of $p(B^+)$ and compute $<D^2>$ :

* use Isotonic calibration (generalization of bins fitting by linear function) - piecewise-constant monotonic function
* randomly divide events into two parts (1-train, 2-calibrate)
* Isotonic fitting on train and $<D^2>$ computation on test
* take mean and std for computed $<D^2>$

$<D^2>$  formula for sample:
$$<D^2> =  \frac{\sum_i[2(p^{mistag}_i - 0.5)]^2 * sw_i}{\sum_i sw_i}
=  \frac{\sum_i[2(p_i(B^+) - 0.5)]^2 * sw_i}{\sum_i sw_i}$$

Formula is symmetric and it is not necessary to compute mistag probability

## Full estimation of systematic error

* set random state
* train the best model (track and vertex taggers with 2-folding with fixed random state)
* do calibration for track and vertex taggers with 2-folding with fixed random state
* compute $p(B^+)$
* do calibration with isotonic 2-folding (random state is fixed)
* compute $<D^2>$

This procedure is repeated (from the scratch) for 30 different random states and then we compute **mean** and **std** for these 30 values of $<D^2>$.